# Title

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

crimeHousing = pd.read_csv("crime-housing-austin-2015.csv")

## Analysis 1

## Analysis 2

## Analysis 3

## Analysis 4

## Analysis 5

## Analysis 6

## Analysis 7

## Analysis 8